In [ ]:
import streamlit as st
from PIL import Image
import pyttsx3
import pytesseract
import os
from langchain_google_genai import GoogleGenerativeAI

# Set up Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Initialize Text-to-Speech engine
engine = pyttsx3.init()

# Initialize Google Generative AI LLM
GEMINI_API_KEY = "Your-API-Key-Here"  # Replace with your API key
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
llm = GoogleGenerativeAI(model="gemini-1.5-pro", api_key=GEMINI_API_KEY)

# App Layout and Design
st.set_page_config(page_title="Aid", layout="centered", page_icon="")
st.title("Aid - AI Assistive Tool")
st.markdown("*Helping visually impaired users with AI-powered image analysis.*")

# Sidebar
st.sidebar.header("Features")
st.sidebar.markdown(
    """
- *🔍 Describe Scene*: AI-powered insights about uploaded images.
- *📝 Extract Text*: OCR to read text in images.
- *🔊 Text-to-Speech*: Listen to the extracted text.
"""
)

st.sidebar.info("Upload an image to get started!")

# File uploader for user input
uploaded_file = st.file_uploader(
    "Upload an image (JPG, PNG, JPEG)", type=["jpg", "jpeg", "png"]
)

# Helper Functions
def extract_text_from_image(image):
    """Extract text using Tesseract OCR."""
    return pytesseract.image_to_string(image)

def text_to_speech(text):
    """Convert extracted text to speech."""
    engine.say(text)
    engine.runAndWait()

def generate_scene_description(prompt, image_data):
    """Generate a scene description using Google Gemini API."""
    full_prompt = f"{prompt}\n\nHere is the image data: {image_data[:500]}..."
    response = llm.predict(text=full_prompt)
    return response

# Process Uploaded Image
if uploaded_file:
    image = Image.open(uploaded_file)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Convert uploaded file into bytes for LLM processing
    image_bytes = uploaded_file.read()

    # Features Buttons
    col1, col2, col3 = st.columns(3)

    # Scene Description
    if col1.button("🔍 Describe Scene"):
        with st.spinner("Generating scene description..."):
            try:
                input_prompt = """
                You are an AI assistant helping visually impaired individuals by describing the scene in the uploaded image. 
                Provide:
                1. A list of visible items with their purposes.
                2. An overall description of the image.
                3. Safety precautions or tips for navigating the scene.
                """
                scene_description = generate_scene_description(input_prompt, image_bytes)
                st.subheader("🔍 Scene Description")
                st.write(scene_description)
            except Exception as e:
                st.error(f"Error generating scene description: {e}")

    # Text Extraction
    if col2.button("📝 Extract Text"):
        with st.spinner("Extracting text from the image..."):
            text = extract_text_from_image(image)
            if text.strip():
                st.subheader("📝 Extracted Text")
                st.text_area("Text Content", text, height=200)
            else:
                st.warning("No text detected in the image.")

    # Text-to-Speech
    if col3.button("🔊 Text-to-Speech"):
        with st.spinner("Converting text to speech..."):
            text = extract_text_from_image(image)
            if text.strip():
                text_to_speech(text)
                st.success("✅ Text-to-Speech conversion completed!")
            else:
                st.warning("No text available for conversion.")

# Footer
st.markdown("---")
st.markdown(
    """
© 2024 VisionAid | Built with ❤ using Streamlit, Google Generative AI, and Python.  
Empowering accessibility through technology.
"""
)
streamlit==1.18.0
Pillow==9.4.0
pyttsx3==2.90
pytesseract==0.3.10
google-generativeai==0.0.3
langchain-google-genai==0.0.1